In [2]:
!pip install gensim
!pip install spacy
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 15.9 MB/s eta 0:00:00


In [3]:
# Data Handling & Plotting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import time

# NLTK for preprocessing
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

# Scikit-learn for Vectorization, Classification, Clustering, and Evaluation
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report, confusion_matrix, silhouette_score
from sklearn.metrics import ConfusionMatrixDisplay

# Gensim for Topic Modeling
import gensim
from gensim import corpora
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel

# spaCy for NER
import spacy
from spacy import displacy

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd

# df = pd.read_csv('youtoxic_english_1000.csv') # ใช้อันนี้ถ้าทำในvs code
df = pd.read_csv('/content/drive/MyDrive/youtoxic_english_1000.csv')
print("DataFrame created:")
display(df.head())

DataFrame created:


,CommentId,VideoId,Text,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsNationalist,IsSexist,IsHomophobic,IsReligiousHate,IsRadicalism
0,Ugg2KwwX0V8-aXgCoAEC,04kJtp6pVXI,If only people would just take a step back and...,False,False,False,False,False,False,False,False,False,False,False,False
1,Ugg2s5AzSPioEXgCoAEC,04kJtp6pVXI,Law enforcement is not trained to shoot to app...,True,True,False,False,False,False,False,False,False,False,False,False
2,Ugg3dWTOxryFfHgCoAEC,04kJtp6pVXI,\nDont you reckon them 'black lives matter' ba...,True,True,False,False,True,False,False,False,False,False,False,False
3,Ugg7Gd006w1MPngCoAEC,04kJtp6pVXI,There are a very large number of people who do...,False,False,False,False,False,False,False,False,False,False,False,False
4,Ugg8FfTbbNF8IngCoAEC,04kJtp6pVXI,"The Arab dude is absolutely right, he should h...",False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
print(df['Text'].iloc[3])

In [6]:
nltk.download('stopwords')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
stop_words.remove('not')
import contractions

def normalize_apostrophe(text):
    text = text.replace("\n", " ")
    text = text.replace("’", "'")
    text = text.replace("`", "'")
    text = text.replace("‘", "'")
    text = re.sub(r"\b[Dd]ont\b", "don't", text)
    return text

def get_wordnet_pos_hw(treebank_tag):
    if treebank_tag.startswith('J'): return wordnet.ADJ
    elif treebank_tag.startswith('V'): return wordnet.VERB
    elif treebank_tag.startswith('N'): return wordnet.NOUN
    elif treebank_tag.startswith('R'): return wordnet.ADV
    else: return wordnet.NOUN

def preprocess_text_hw(text):
    # text = contractions.fix(text)
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    clean_tokens = []
    for word, tag in pos_tags:
        word = re.sub(r'[^a-zA-Z]', '', word).lower()
        if word and word not in stop_words and len(word) > 2:
            word = lemmatizer.lemmatize(word, pos=get_wordnet_pos_hw(tag))
            clean_tokens.append(word)
    return ' '.join(clean_tokens)


df['Text'] = df['Text'].apply(normalize_apostrophe)
df['Text'] = df['Text'].astype(str).apply(contractions.fix)
df['Text'] = df['Text'].apply(preprocess_text_hw)
display(df.head())
print(df['Text'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


,CommentId,VideoId,Text,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsNationalist,IsSexist,IsHomophobic,IsReligiousHate,IsRadicalism
0,Ugg2KwwX0V8-aXgCoAEC,04kJtp6pVXI,people would take step back not make case not ...,False,False,False,False,False,False,False,False,False,False,False,False
1,Ugg2s5AzSPioEXgCoAEC,04kJtp6pVXI,law enforcement not train shoot apprehend trai...,True,True,False,False,False,False,False,False,False,False,False,False
2,Ugg3dWTOxryFfHgCoAEC,04kJtp6pVXI,not reckon black live matter banner hold white...,True,True,False,False,True,False,False,False,False,False,False,False
3,Ugg7Gd006w1MPngCoAEC,04kJtp6pVXI,large number people not like police officer ca...,False,False,False,False,False,False,False,False,False,False,False,False
4,Ugg8FfTbbNF8IngCoAEC,04kJtp6pVXI,arab dude absolutely right not shot extra time...,False,False,False,False,False,False,False,False,False,False,False,False


0      people would take step back not make case not ...
1      law enforcement not train shoot apprehend trai...
2      not reckon black live matter banner hold white...
3      large number people not like police officer ca...
4      arab dude absolutely right not shot extra time...
                             ...                        
995                       remember send national defence
996    stats not represent problem race baiting attit...
997                   quote mother wow hit hard accurate
998                                         video racist
999                              god narrator annoy lisp
Name: Text, Length: 1000, dtype: object


In [ ]:
# from gensim.models import KeyedVectors
# from gensim.models import Word2Vec

In [ ]:
# import gensim.downloader as api
# fasttext = api.load("fasttext-wiki-news-subwords-300") อันนี้ word embedding

In [ ]:
# def sentence_embedding(sentence):
#     words = sentence.split()อันนี้ของ word embedding
#     vectors = []

#     for word in words:
#         vectors.append(fasttext[word])

#     if len(vectors) == 0:
#         return np.zeros(fasttext.vector_size)

#     return np.mean(vectors, axis=0)

In [ ]:
# X_embedded = np.array([sentence_embedding(text) for text in df["Text"]])

In [ ]:
print(df['Text'].iloc[3])

In [8]:
All = [
    "IsToxic","IsAbusive","IsThreat","IsProvocative",
    "IsObscene","IsHatespeech","IsRacist","IsNationalist",
    "IsSexist","IsHomophobic","IsReligiousHate","IsRadicalism"
]
label_sums = df[All].sum()
print(label_sums)

IsToxic            462
IsAbusive          353
IsThreat            21
IsProvocative      161
IsObscene          100
IsHatespeech       138
IsRacist           125
IsNationalist        8
IsSexist             1
IsHomophobic         0
IsReligiousHate     12
IsRadicalism         0
dtype: int64


In [9]:
df["IsNeutral"] = (df[All].sum(axis=1) == 0).astype(int)
print("Neutral samples:", df["IsNeutral"].sum())

Neutral samples: 538


In [ ]:
df["IsSumRacist"] = (
    df["IsReligiousHate"] |
    df["IsHatespeech"] |
    df["IsRacist"]
).astype(int)

In [10]:
label_cols = [
    "IsToxic",
    "IsAbusive",
    "IsNeutral",
]
df[label_cols] = df[label_cols].fillna(0).astype(int)

y = df[label_cols].values
print(df[label_cols].dtypes)

IsToxic      int64
IsAbusive    int64
IsNeutral    int64
dtype: object


In [11]:
label_sums = df[label_cols].sum()
print(label_sums)

IsToxic      462
IsAbusive    353
IsNeutral    538
dtype: int64


In [12]:
vectorizer = TfidfVectorizer(
    min_df=3,
    max_df=0.8,
    ngram_range=(1,2),
    stop_words="english",
    max_features=20000
)

tfidf_matrix = vectorizer.fit_transform(df['Text'])
feature_names = vectorizer.get_feature_names_out()

print(f"Created a TF-IDF matrix of shape: {tfidf_matrix.shape}")
print(f"(Documents, Features/Vocabulary)")
print(f"Vocabulary size: {len(feature_names)}")
print(f"Sample features: {feature_names[500:510]}")

Created a TF-IDF matrix of shape: (1000, 1136)
(Documents, Features/Vocabulary)
Vocabulary size: 1136
Sample features: ['job' 'joe' 'joe rogan' 'joke' 'judge' 'jury' 'justice' 'justify'
 'justify murder' 'kid']


In [13]:
X = tfidf_matrix
y = df[label_cols].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.4,
    random_state=42,
)

X_test, X_val, y_test, y_val = train_test_split(
    X_test, y_test,
    test_size=0.5,
    random_state=42,
)

print(f"X shape: {X.shape}")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"X_train shape: {X_val.shape}")

X shape: (1000, 1136)
X_train shape: (600, 1136)
X_test shape: (200, 1136)
X_train shape: (200, 1136)


**OneVsRestClassifier(LogisticRegression):Baseline**

In [15]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.linear_model import LogisticRegression

model = OneVsRestClassifier(
    LogisticRegression(
        max_iter=3000,
        class_weight="balanced"
    )
)

model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
y_val_pred = model.predict(X_val)

print("Test Accuracy:", accuracy_score(y_test, y_test_pred))
print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))
print("Test Accuracy:", f1_score(y_test, y_test_pred, average="micro"))
print("Validation Accuracy:", f1_score(y_val, y_val_pred, average="micro"))

print(classification_report(y_test, y_val_pred, target_names=label_cols))

Test Accuracy: 0.59
Validation Accuracy: 0.685
Test Accuracy: 0.6578947368421053
Validation Accuracy: 0.7177570093457943
              precision    recall  f1-score   support

     IsToxic       0.60      0.42      0.49       107
   IsAbusive       0.42      0.38      0.40        73
   IsNeutral       0.50      0.68      0.58        93

   micro avg       0.51      0.50      0.50       273
   macro avg       0.51      0.49      0.49       273
weighted avg       0.52      0.50      0.50       273
 samples avg       0.51      0.53      0.51       273

